In [2]:
import pandas as pd
import pickle
import numpy as np
import plotly.express as px

In [3]:
statistics_db = pd.read_parquet("/home/imgyuri/BIOINFO1/Bioinfo1_Labsession/FinalProject/Statistics_result.parquet")
statistics_db = statistics_db.reset_index(drop=True)

print(statistics_db.head(10))
print(len(statistics_db))


       GOterm  GeneNumber       ClipPval  ClipFoldChange     RNAdenPval  \
0  GO:0005634        2741   3.243718e-12       -0.330358  1.238783e-110   
1  GO:0005737        2589   9.917849e-17       -0.309420   2.110516e-77   
2  GO:0016020        1507   1.073162e-59        0.635663  8.458918e-200   
3  GO:0016021        1307  2.094352e-122        0.932191   0.000000e+00   
4  GO:0000166        1168   2.729137e-02       -0.084324   5.617830e-30   
5  GO:0005739         910   1.567666e-04       -0.279657   1.329863e-03   
6  GO:0005886         766   2.745082e-10        0.390605   6.066301e-19   
7  GO:0003677         688   2.522728e-03       -0.287970   2.245808e-29   
8  GO:0005783         510   1.311150e-36        0.830254   1.426313e-86   
9  GO:0005829         486   1.529188e-08       -0.430284   3.304010e-11   

   RNAdenFoldChange  
0         -0.353345  
1         -0.296261  
2          0.510233  
3          0.726462  
4         -0.238740  
5          0.026959  
6          0.246381 

In [4]:
GOResultToPlot = statistics_db[(statistics_db['ClipPval'] <= 0.05) & (statistics_db['RNAdenPval'] <= 0.05) & (statistics_db['GeneNumber']>=10)]
GOResultToPlot['log_ClipPval'] = np.log10(GOResultToPlot['ClipPval'])
GOResultToPlot['log_RNAdenPval'] = np.log10(GOResultToPlot['RNAdenPval'])
print(len(GOResultToPlot))

57


/tmp/ipykernel_78453/4179151703.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GOResultToPlot['log_ClipPval'] = np.log10(GOResultToPlot['ClipPval'])
/home/imgyuri/miniconda3/envs/chem/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/tmp/ipykernel_78453/4179151703.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GOResultToPlot['log_RNAdenPval'] = np.log10(GOResultToPlot['RNAdenPval'])


In [5]:
GOResultToPlot.to_csv("/home/imgyuri/BIOINFO1/Bioinfo1_Labsession/FinalProject/GOresultToPlot_filter_controlonly.csv")

In [6]:
GO_Notinterest = {
    'GO:0005829': 'cytosol',
    'GO:0005654' : 'nucleoplasm',
    'GO:0005886' : 'plasma membrane',
    'GO:0016020' : 'membrane',
    'GO:0016021' : 'membrane'
   # 'GO:0005789' : 'endoplasmic reticulum membrane', # exclude outlier
}
# Exclude GO terms that are not interesting(GO_Notinterest.keys()) in GOResultToPlot['GOTerm']
GOResultToPlot = GOResultToPlot[~GOResultToPlot['GOterm'].isin(GO_Notinterest.keys())]
GOResultToPlot.sort_values(by='GeneNumber', ascending=True, inplace=True)

In [7]:
GOtoAnnotate = {
    'GO:0005783': 'endoplasmic reticulum',
    'GO:0005788': 'endoplasmic reticulum lumen',
    'GO:0005789': 'endoplasmic reticulum membrane',
    'GO:0005794': 'Golgi apparatus',
    'GO:0009986': 'cell surface',
    'GO:0005739': 'mitochondrion',
    'GO:0005509': 'calcium ion binding',
    'GO:0005634': 'nucleus',
    'GO:0005737': 'cytoplasm',
    'GO:0005576': 'extracellular region',
    'GO:0016020': 'membrane',
    'GO:0005576': 'extracellular region',
    'GO:0030203':'glycosaminoglycan metabolic process',
    'GO:0006334': 'nucleosome assembly',
    'GO:0000786': 'nucleosome',
}

In [8]:
Interest_df = GOResultToPlot[GOResultToPlot['GOterm'].isin(GOtoAnnotate.keys())]
print(Interest_df)

        GOterm  GeneNumber      ClipPval  ClipFoldChange     RNAdenPval  \
32  GO:0005788          46  1.035078e-12        1.243625   4.349346e-14   
23  GO:0009986          88  7.156970e-12        1.012361   4.000938e-16   
16  GO:0005509         177  3.403011e-06        0.610236   8.109868e-08   
13  GO:0005576         226  1.995155e-12        0.602593   8.416354e-48   
11  GO:0005789         292  1.453983e-27        0.926719   3.682884e-62   
10  GO:0005794         453  1.224669e-08        0.369477   1.414970e-13   
8   GO:0005783         510  1.311150e-36        0.830254   1.426313e-86   
5   GO:0005739         910  1.567666e-04       -0.279657   1.329863e-03   
1   GO:0005737        2589  9.917849e-17       -0.309420   2.110516e-77   
0   GO:0005634        2741  3.243718e-12       -0.330358  1.238783e-110   

    RNAdenFoldChange  log_ClipPval  log_RNAdenPval  
32          0.660972    -11.985027      -13.361576  
23          0.530367    -11.145271      -15.397838  
16          0.2

In [9]:
# Interest_df = GOResultToPlot[GOResultToPlot['GOterm'].isin(GOtoAnnotate.keys())]
# print(Interest_df)

In [10]:
def format_number_scientific(num):
    base, exponent = f"{num:.1e}".split('e')
    exponent = int(exponent)
    # Convert the exponent to superscript
    superscript_map = str.maketrans('-0123456789', '⁻⁰¹²³⁴⁵⁶⁷⁸⁹')
    exponent_str = str(exponent).translate(superscript_map)
    return f"{base}x10{exponent_str}"


In [81]:
ax_ay = {'GO:0005788': [150, -30], 
                  'GO:0009986': [230, 100], 
                  'GO:0005509': [80, 56], 
                  'GO:0005576': [-100, -60], 
                  'GO:0005789': [200, 25], 
                  'GO:0005794': [40, 100], 
                  'GO:0005783': [30, -70], 
                  'GO:0005739': [-70, -50], 
                  'GO:0005737': [-80, -30], 
                  'GO:0005634': [-30, 50]}

In [84]:
fig=px.scatter(GOResultToPlot, x='ClipFoldChange', y="RNAdenFoldChange", 
               size="GeneNumber", color = 'log_ClipPval', 
               color_continuous_scale='YlOrRd_r'
               ,template='simple_white', size_max=40,
               hover_data=['GOterm', 'GeneNumber', 'ClipFoldChange', 'RNAdenFoldChange', 'ClipPval', 'RNAdenPval'],)

fig.update_layout(
    coloraxis_colorbar=dict(
        title=dict(
            text="Term-specific enrichment confidence (false discovery rate)",
            side="right"
        ),
        ticktext=[f"10<sup>{-i}</sup>" for i in range(0, 21, 2)],
        len=1.1,
        thickness=20,
    ),
    width=1000, height=500,
    font=dict(
        size=10
    )
)
fig.update_xaxes(range=[-2.0, 5.0],griddash='dash', tickvals=list(range(-2, 6)), tickmode="linear", dtick=0.5, title_text='Enrichment level of LIN28A-bound CLIP tags (log<sub>2</sub>)',
                 showgrid=True, showline=True)
fig.update_yaxes(range=[-1.0, 1.5],griddash='dash', tickvals=[-1.0, -0.5, 0.0, 0.5, 1.0, 1.5], tickmode="linear", dtick=0.5, title_text='Ribisome density change upon <i>Lin28a</i> knockdown (log<sub>2</sub>)',
                 showgrid=True, showline=True)

# annotation
offsets = {
    "x": [-0.1, 0.1, -0.1, 0.1],
    "y": [-0.1, 0.1, 0.1, -0.1]
}
offset_idx = 0
for i, row in GOResultToPlot.iterrows():
    if row['GOterm'] in GOtoAnnotate:
        fig.add_annotation(x=GOResultToPlot['ClipFoldChange'][i], y=GOResultToPlot['RNAdenFoldChange'][i], 
                           text=f"{GOtoAnnotate[row['GOterm']]}({row['GeneNumber']})<br> C={format_number_scientific(row['ClipPval'])}, R={format_number_scientific(row['RNAdenPval'])}",
                           showarrow=True,
                           bordercolor='black',
                           borderwidth=1,
                           ax=ax_ay[row['GOterm']][0], ay=ax_ay[row['GOterm']][1])
        offset_idx += 1
fig.show()